In [8]:
import re
import requests
import pandas as pd
import psqlite as pql
import numpy as np
import time
import statistics
import json

from collections import defaultdict
from datetime import datetime, timedelta, date

from run_query import run_query

# read from csv
element_liquidity = pd.read_csv("./element_liquidity.csv")
element_transfers = pd.read_csv("./element_transfers.csv")

# fix usd value
element_liquidity["price"] = (
    element_liquidity["deposit_size_base_usd"] / element_liquidity["deposit_size_base"]
)
element_liquidity["deposit_size_base_usd"] = (
    element_liquidity["lp_tokens_acquired"] * element_liquidity["price"]
)


In [6]:
# display(element_liquidity.head(1))
display(sum(element_liquidity.liquidity_provider=='\\xece48d83a2f89b6f5ded23362ac15c19f91e0ddb'))
display(sum(element_liquidity.liquidityProvider=='\\xece48d83a2f89b6f5ded23362ac15c19f91e0ddb'))
display(sum(element_transfers.sender=='\\xece48d83a2f89b6f5ded23362ac15c19f91e0ddb'))
display(element_transfers.loc[element_transfers.sender=='\\xece48d83a2f89b6f5ded23362ac15c19f91e0ddb',:])
display(element_transfers.loc[element_transfers.sender=='\\xece48d83a2f89b6f5ded23362ac15c19f91e0ddb','tokens_transferred'].values==0)
display(sum(element_transfers.recipient=='\\xece48d83a2f89b6f5ded23362ac15c19f91e0ddb'))

display(element_transfers.shape)
# remove user who sends before receiving ()
element_transfers = element_transfers.loc[element_transfers.sender!='\\xece48d83a2f89b6f5ded23362ac15c19f91e0ddb',:]
display(element_transfers.shape)

0

0

1

,block_time,block_number,contract_address,sender,recipient,tokens_transferred,tx_hash,index,tx_index
4302,2022-09-23 07:21:11+00:00,15594519,\xa2b3d083aa1eaa8453bfb477f062a208ed85cbbf,\xece48d83a2f89b6f5ded23362ac15c19f91e0ddb,\x5750b90fa979aa4b8a28e05f865a66c0be396750,24.397053,\x35cab14f709bb6100f47d4448096fdb36b53441c6142...,15.0,21.0


array([False])

0

(4554, 9)

(4553, 9)

In [9]:
display(element_transfers.head(1))
# element_transfers.token.value_counts()
# from original query:
# --Exclude mints and burns by the Balancer contract
# where "sender" not in ('\x0000000000000000000000000000000000000000', '\xba12222222228d8ba445958a75a0704d566bf2c8') 
# and "recipient" not in ('\x0000000000000000000000000000000000000000', '\xba12222222228d8ba445958a75a0704d566bf2c8')
eliminate_addresses = ['\\x0000000000000000000000000000000000000000', '\\xba12222222228d8ba445958a75a0704d566bf2c8']
sender_bad = element_transfers.sender.isin(eliminate_addresses)
print(f"txn's where sender is 0 address or balancer: {sum(sender_bad)}")
recipient_bad = element_transfers.recipient.isin(eliminate_addresses)
print(f"txn's where recipient is 0 address or balancer: {sum(recipient_bad)}")
amount_bad = element_transfers.tokens_transferred==0
print(f"txn's where tokens_transferred is 0 : {sum(amount_bad)}")
print(f"sum of the above three: {sum(sender_bad)+sum(recipient_bad)+sum(amount_bad)}")
print(f"combined txn's: {sum(sender_bad | recipient_bad | amount_bad)}")


,block_time,block_number,contract_address,sender,recipient,tokens_transferred,tx_hash,index,tx_index
0,2021-08-06 04:15:52+00:00,12969195,\x787546bf2c05e3e19e2b6bde57a203da7f682eff,\xf83895c42ea4c2b6e1f597828a1f27a0d7147497,\xa500e3a6ec04f77a833922f257bdaf2b0c47924d,103526.225703,\x96d9e74e6973eb26ce9475a4dadd5c580f655abe038a...,501.0,315.0


txn's where sender is 0 address or balancer: 0
txn's where recipient is 0 address or balancer: 0
txn's where tokens_transferred is 0 : 0
sum of the above three: 0
combined txn's: 0


In [45]:
element_transfers_filtered = element_transfers.loc[~(sender_bad | recipient_bad | amount_bad), :]
display(element_transfers_filtered.shape)
element_transfers_filtered.to_csv("element_transfers.csv", index=False)

(1762, 10)

In [4]:
# for _, row in lp_events.iterrows():
#     new_row = dict(row)
#     token_name = new_row["token_name"]
#     if token_name is None:
#         # display("warning: none token name:")
#         # display(new_row)
#         none_tx_hashes.append(new_row["tx_hash"])
# display(lp_events.loc[lp_events.tx_hash.isin(none_tx_hashes),:])

for _, row in element_transfers.iterrows():
    if row["token"] is None:
        print(row)

In [5]:
# get token names
token_names = pd.read_csv("element_tokens.csv")
display(token_names)


,token_address_raw,token_name,long_name
0,\x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,Wrapped BTC
1,\x43b4fdfd4ff969587185cdb6f0bd875c5fc83f8c,alUSD3CRV-f,Curve.fi Factory USD Metapool: Alchemix USD
2,\x5a6a4d54456819380173272a5e8e9b9904bdf41b,MIM-3LP3CRV-f,Curve.fi Factory USD Metapool: Magic Internet ...
3,\xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin
4,\xed279fdd11ca84beef15af5d39bb4d4bee23f0ca,LUSD3CRV-f,Curve.fi Factory USD Metapool: Liquity
...,...,...,...
95,\x5746afd392b13946aacbda40317751db27d8b918,LPePyvUSDC-24FEB23,LP Element Principal Token yvUSDC-24FEB23
96,\x724e3073317d4b1a8d0c6e89b137ea5af1f4051e,ePyvCurve-stETH-24FEB23,Element Principal Token yvCurve-stETH-24FEB23
97,\x07f589ea6b789249c83992dd1ed324c3b80fd06b,LPePyvCurve-stETH-24FEB23,LP Element Principal Token yvCurve-stETH-24FEB23
98,\x7e4fee0c14673db4f1526fdbea53819e5808a9b5,ePyvBalancer-BoostedAaveUSD-04MAY23,Element Principal Token yvBalancer-BoostedAave...


In [6]:
# token mappings
contract_url = "https://raw.githubusercontent.com/delvtech/elf-deploy/main/addresses/mainnet.json"
element_token_mappings = requests.get(contract_url).json()


In [17]:
tokens_from_github = []
for token, data in element_token_mappings["tranches"].items():
    for tranche in data:
        for token_type in ("ptPool", "ytPool"):
            if token_type in tranche:
                tokens_from_github.append(
                    {
                        "expiry_timestamp": tranche["expiration"],
                        "expiry_datetime": datetime.fromtimestamp(
                            tranche["expiration"]
                        ).strftime("%Y-%m-%d %H:%M:%S"),
                        "token_address_raw": "\\"
                        + tranche[token_type]["address"][1:].lower(),
                        "token_type": "LPeP" if token_type == "ptPool" else "LPeY",
                        "poolId": "\\" + tranche[token_type]["poolId"][1:].lower(),
                    }
                )
tokens_from_github = pd.DataFrame(tokens_from_github)
token_expiry = pd.merge(tokens_from_github, token_names, on="token_address_raw")
token_expiry = token_expiry.loc[:, ["token_address_raw", "token_name", "expiry_timestamp", "expiry_datetime"]]
display(token_expiry.head(1))
pql.register(token_expiry, "token_expiry")

,token_address_raw,token_name,expiry_timestamp,expiry_datetime
0,\xc9ad279994980f8df348b526901006972509677f,LPePyvCurve-alUSD-28JAN22,1643382460,2022-01-28 10:07:40


In [18]:
element_liquidity.lp_token.isin(token_expiry.token_name).value_counts()

True     28821
False       40
Name: lp_token, dtype: int64

In [19]:
element_liquidity.loc[~element_liquidity.lp_token.isin(token_expiry.token_name),:]
element_liquidity.loc[~element_liquidity.lp_token.isin(token_expiry.token_name),'lp_token'].value_counts()
# rename liquidity data to match onchain token names
old_names = ['LPePyvcrvSTETH-16SEP22', 'LPePyvcrvSTETH-24FEB23', 'LPePyvCurveLUSD-16SEP22', 'LPePyvcrv3crypto-16SEP22']
new_names = ['LPePyvCurve-stETH-16SEP22', 'LPePyvCurve-stETH-24FEB23', 'LPePyvCurve-LUSD-16SEP22', 'LPePyvCurve-3Crypto-16SEP22']
for idx in range(len(old_names)):
    rows_to_replace = element_liquidity.lp_token==old_names[idx]
    element_liquidity.loc[rows_to_replace, 'lp_token'] = new_names[idx]
    print(f"replaced {sum(rows_to_replace):2g} rows for {old_names[idx]:24s} with {new_names[idx]:27s}")

replaced 23 rows for LPePyvcrvSTETH-16SEP22   with LPePyvCurve-stETH-16SEP22  
replaced  9 rows for LPePyvcrvSTETH-24FEB23   with LPePyvCurve-stETH-24FEB23  
replaced  4 rows for LPePyvCurveLUSD-16SEP22  with LPePyvCurve-LUSD-16SEP22   
replaced  4 rows for LPePyvcrv3crypto-16SEP22 with LPePyvCurve-3Crypto-16SEP22


In [20]:
element_transfers.token.isin(token_expiry.token_name).value_counts()

True    1765
Name: token, dtype: int64

In [21]:
# add to pql
pql.register(element_liquidity, "liquidity")
pql.register(element_transfers, "transfers")

# the `lp_events` table contains all events related to Element LP tokens
# and contains expiry metadata for each related token

lp_events = pql.q(
    """
     
SELECT ROW_NUMBER() OVER (PARTITION BY address, token_expiry.token_name
                          ORDER BY datetime ASC) AS row_num,
       DATETIME_PARSE(datetime) AS datetime,
       tx_hash,
       address, -- signer: et."from"
       liquidityProvider,
       token_expiry.token_name AS token_name,
       token_change,
       usd_change,
       recipient_address,
       event_type,
       expiry_timestamp,
       DATETIME(expiry_timestamp, 'unixepoch') AS expiry_datetime,
       evt_index,
       tx_index

 FROM (

    SELECT evt_block_time AS datetime,
           evt_tx_hash AS tx_hash,
           liquidity_provider AS address,
           liquidityProvider,
           lp_token,
           lp_tokens_acquired AS token_change,
           deposit_size_base_usd AS usd_change,
           '' AS recipient_address,
           'deposit' AS event_type,
           "index" AS evt_index,
           tx_index
      FROM liquidity
     WHERE lp_tokens_acquired >= 0

     UNION ALL

    SELECT evt_block_time AS datetime,
           evt_tx_hash AS tx_hash,
           liquidity_provider AS address,
           liquidityProvider,
           lp_token,
           lp_tokens_acquired AS token_change,
           deposit_size_base_usd AS usd_change,
           '' AS recipient_address,
           'withdraw' AS event_type,
           "index" AS evt_index,
           tx_index
      FROM liquidity
     WHERE lp_tokens_acquired < 0

     UNION ALL

    SELECT block_time AS datetime,
           tx_hash AS tx_hash,
           sender AS address,
           '' AS liquidityProvider,
           token AS lp_token,
           -tokens_transferred AS token_change,
           0 AS usd_change,
           recipient AS recipient_address,
           'send' AS event_type,
           "index" AS evt_index,
           tx_index
      FROM transfers
     WHERE token LIKE 'LP%'

) a LEFT JOIN token_expiry
      ON a.lp_token = token_expiry.token_name

-- note that tie-breaking sorting using the `evt_index`
-- resolves cases where smart contracts may receive funds
-- and then disburse them in a single transaction
ORDER BY datetime ASC, tx_index ASC, evt_index ASC

"""
)
pql.register(lp_events, "lp_events")


In [22]:

# list all contracts where signer isn't same as sender
listOfDoubleAddresses = list(
    set(
        (
            lp_events.loc[
                lp_events.address != lp_events.liquidityProvider, ["liquidityProvider"]
            ]
        ).liquidityProvider.values
    )
)
listOfDoubleAddresses = listOfDoubleAddresses[1:]

# replace signer with sender for these contracts
n = 0
for double_address in listOfDoubleAddresses:
    idxAffectedRows = lp_events.liquidityProvider == double_address
    different_rows = sum(
        lp_events.loc[idxAffectedRows, "address"]
        != lp_events.loc[idxAffectedRows, "liquidityProvider"]
    )
    n += different_rows
    print(
        f"overwriting liquidity_provider address for {double_address}"
        f"for {different_rows} rows"
    )
    lp_events.loc[idxAffectedRows, "address"] = lp_events.loc[
        idxAffectedRows, "liquidityProvider"
    ]
print(f"total affected rows replaced: {n}. nice!")

# what's the most recent timestamp in the LP data? used later
max_datetime = pql.q("SELECT MAX(datetime) FROM lp_events")["MAX(datetime)"][0]

# helper function to look up previous balance
def get_usd_balance_for_address(address_index, address):
    """
    Note that since this is based off of the 'rolling_usd_balance' it
    includes the EARLY_BIRD_BONUS.
    """
    usd_balance_total = 0
    for token_name in address_index[address].keys():
        usd_balance_total += address_index[address][token_name][-1][
            "rolling_usd_balance"
        ]
    return usd_balance_total


def build_token_index(lp_events):
    """
    USD credit for LPs is given at the time of deposit
    i.e., the ETH price at the time of deposit is how their "credit"
    for the airdrop is calculated

    when LP tokens are transferred between addresses, we must retain
    the "credit value" of the tokens even upon transfer
    """

    EARLY_BIRD_BONUS = 1.0  # 0%
    EARLY_BIRD_USD_THRESHOLD = 0 * 1e6  # 0 million
    EARLY_BIRD_TIME_DELTA = timedelta(days=0)  # 0 days

    # for storing full rows for a given address, token pair
    address_token_index = defaultdict(dict)
    # for storing the cumulative deposits (and not withdrawals) to a pool
    pool_index = {}

    # for storing full rows for a given address, token pair
    address_token_index = defaultdict(dict)
    # for storing the cumulative deposits (and not withdrawals) to a pool
    pool_index = {}
    # for storing bad addresses to be investigated later (withdraw before deposit)
    bad_address = []
    bad_address_zero_balance = []

    # iterate according to:
    # ORDER BY datetime ASC, tx_index ASC, evt_index ASC
    for _, row in lp_events.iterrows():
        new_row = dict(row)

        address = new_row["address"]
        event_type = new_row["event_type"]
        token_name = new_row["token_name"]
        if token_name is None:
            display("warning: none token name:")
            display(new_row)
        token_change = new_row["token_change"]
        usd_change = new_row["usd_change"]
        event_datetime = new_row["datetime"]
        recipient_address = new_row["recipient_address"]

        # membership in `address_token_index`
        if not token_name in address_token_index[address]:

            if event_type == "deposit":

                # update cumulative token deposits
                if token_name not in pool_index:
                    apply_early_bird_bonus = True
                    pool_index[token_name] = {}
                    pool_index[token_name]["usd_balance"] = usd_change
                    pool_index[token_name][
                        "first_deposit_datetime"
                    ] = datetime.strptime(event_datetime, "%Y-%m-%d %H:%M:%S")
                else:
                    elapsed_time = (
                        datetime.strptime(event_datetime, "%Y-%m-%d %H:%M:%S")
                        - pool_index[token_name]["first_deposit_datetime"]
                    )
                    if (
                        pool_index[token_name]["usd_balance"]
                        <= EARLY_BIRD_USD_THRESHOLD
                        and elapsed_time < EARLY_BIRD_TIME_DELTA
                    ):
                        apply_early_bird_bonus = True
                    else:
                        apply_early_bird_bonus = False
                    pool_index[token_name]["usd_balance"] += usd_change

                new_row["cumulative_deposits_for_pool"] = pool_index[token_name][
                    "usd_balance"
                ]
                new_row["first_deposit_for_pool_datetime"] = pool_index[token_name][
                    "first_deposit_datetime"
                ].strftime("%Y-%m-%d %H:%M:%S")

                new_row["rolling_token_balance"] = token_change
                new_row["rolling_usd_balance"] = (
                    usd_change * EARLY_BIRD_BONUS
                    if apply_early_bird_bonus
                    else usd_change
                )

                # need get_usd_balance lookup here: just because it's their first transaction for this token, doesn't mean it's their first transaction overall
                new_row[
                    "all_pool_usd_balance_for_address"
                ] = get_usd_balance_for_address(address_token_index, address) + (
                    usd_change * EARLY_BIRD_BONUS
                    if apply_early_bird_bonus
                    else usd_change
                )

            elif event_type == "withdraw":
                print("WARNING: withdraw before receive")
                bad_address.append(address)
                print(address)
                continue

            elif event_type == "send":
                # freak out only if it's not the zero address
                if address != "\\x0000000000000000000000000000000000000000":
                    print("WARNING: send before receive")
                    bad_address.append(address)
                    print(address)
                continue  # in all cases continue

            else:
                raise Exception("unknown event_type: {}".format(event_type))

            address_token_index[address][token_name] = [new_row]

        # token has already been traded
        else:

            last_row = address_token_index[address][token_name][-1]

            if event_type == "deposit":

                # update cumulative token deposits
                elapsed_time = (
                    datetime.strptime(event_datetime, "%Y-%m-%d %H:%M:%S")
                    - pool_index[token_name]["first_deposit_datetime"]
                )
                if (
                    pool_index[token_name]["usd_balance"] <= EARLY_BIRD_USD_THRESHOLD
                    and elapsed_time < EARLY_BIRD_TIME_DELTA
                ):
                    apply_early_bird_bonus = True
                else:
                    apply_early_bird_bonus = False
                pool_index[token_name]["usd_balance"] += usd_change

                new_row["rolling_token_balance"] = (
                    last_row["rolling_token_balance"] + token_change
                )
                new_row["rolling_usd_balance"] = last_row["rolling_usd_balance"] + (
                    usd_change * EARLY_BIRD_BONUS
                    if apply_early_bird_bonus
                    else usd_change
                )
                new_row[
                    "all_pool_usd_balance_for_address"
                ] = get_usd_balance_for_address(address_token_index, address) + (
                    usd_change * EARLY_BIRD_BONUS
                    if apply_early_bird_bonus
                    else usd_change
                )

            elif event_type == "withdraw":

                new_row["rolling_token_balance"] = (
                    last_row["rolling_token_balance"] + token_change
                )

                # token change is explicitly negative in a withdraw
                if last_row["rolling_token_balance"] == 0:
                    print("WARNING: token balance is zero")
                    print(last_row)
                    bad_address_zero_balance.append(address)
                    usd_balance_withdrawn = 0
                else:
                    usd_balance_withdrawn = (
                        token_change / last_row["rolling_token_balance"]
                    ) * last_row["rolling_usd_balance"]
                    new_row["usd_change"] = usd_balance_withdrawn  # for tracking only

                # so we add to decrement
                new_row["rolling_usd_balance"] = (
                    last_row["rolling_usd_balance"] + usd_balance_withdrawn
                )

                new_row["all_pool_usd_balance_for_address"] = (
                    get_usd_balance_for_address(address_token_index, address)
                    + usd_balance_withdrawn
                )

            elif event_type == "send":

                # calculate the new token balance since some tokens have been sent
                new_row["rolling_token_balance"] = (
                    last_row["rolling_token_balance"] + token_change
                )

                # calculate the dollar equivalency of the sent tokens
                if last_row["rolling_token_balance"] == 0:
                    print("WARNING: token balance is zero")
                    print(last_row)
                    bad_address_zero_balance.append(address)
                    usd_balance_sent = 0
                else:
                    usd_balance_sent = (
                        token_change / last_row["rolling_token_balance"]
                    ) * last_row["rolling_usd_balance"]
                    new_row["usd_change"] = usd_balance_sent  # for tracking only

                # calculate the new usd balance
                new_row["rolling_usd_balance"] = (
                    last_row["rolling_usd_balance"] + usd_balance_sent
                )
                new_row["all_pool_usd_balance_for_address"] = (
                    get_usd_balance_for_address(address_token_index, address)
                    + usd_balance_sent
                )

                # construct a new row for the 'receive' event
                receiver_row = dict(new_row)
                receiver_row["address"] = new_row["recipient_address"]
                receiver_row["recipient_address"] = ""
                receiver_row["token_change"] = -token_change
                receiver_row["usd_change"] = -usd_balance_sent
                receiver_row["event_type"] = "receive"

                receiver_address = receiver_row["address"]

                # if we've never seen this address receive this token type
                if token_name not in address_token_index[receiver_address]:

                    # set the relevant new balances
                    receiver_row[
                        "rolling_token_balance"
                    ] = -token_change  # TODO: check this sign
                    receiver_row["rolling_usd_balance"] = -usd_balance_sent

                    receiver_row["all_pool_usd_balance_for_address"] = -usd_balance_sent

                    # add the receiver row to its index
                    address_token_index[receiver_address][token_name] = [receiver_row]

                # if we've seen this address receive this token type
                else:

                    # then we have existing knowledge of its balances, which we must update
                    last_receiver_row = address_token_index[receiver_address][
                        token_name
                    ][-1]

                    # TODO: check this sign
                    receiver_row["rolling_token_balance"] = (
                        last_receiver_row["rolling_token_balance"] - token_change
                    )
                    receiver_row["rolling_usd_balance"] = (
                        last_receiver_row["rolling_usd_balance"] - usd_balance_sent
                    )

                    receiver_row["all_pool_usd_balance_for_address"] = (
                        get_usd_balance_for_address(address_token_index, address)
                        - usd_balance_sent
                    )

                    address_token_index[receiver_address][token_name].append(
                        receiver_row
                    )

            # we always track the cumulative deposits into the pool, the first deposit datetime, and append the new row to the index
            new_row["cumulative_deposits_for_pool"] = pool_index[token_name][
                "usd_balance"
            ]
            new_row["first_deposit_for_pool_datetime"] = pool_index[token_name][
                "first_deposit_datetime"
            ].strftime("%Y-%m-%d %H:%M:%S")

            address_token_index[address][token_name].append(new_row)
    return (address_token_index, bad_address, bad_address_zero_balance)

address_token_index, bad_address, bad_address_zero_balance = build_token_index(
    lp_events
)
if len(bad_address_zero_balance) > 0:
    print("bad addresses with zero balance: \n", bad_address_zero_balance)
    pd.DataFrame(bad_address_zero_balance).to_csv(
        "bad_address_zero_balance.csv", index=False, header=False
    )

overwriting liquidity_provider address for \xc01318bab7ee1f5ba734172bf7718b5dc6ec90e1for 1 rows
overwriting liquidity_provider address for \xb3dfe140a77ec43006499cb8c2e5e31975cad909for 9 rows
overwriting liquidity_provider address for \x887d69a7b3dd3e36d7c56556d2df24ee117474f2for 2 rows
overwriting liquidity_provider address for \xbc79855178842fdba0c353494895deef509e26bbfor 3 rows
overwriting liquidity_provider address for \x16e46dec73c0d372b461718869fba264ec08b137for 8 rows
overwriting liquidity_provider address for \x27f0e8beccfa594f4b08c24226ce75c535b900eafor 8 rows
overwriting liquidity_provider address for \x849d52316331967b6ff1198e5e32a0eb168d039dfor 12 rows
overwriting liquidity_provider address for \xcc425c2bb07a5d21956eaed275ba677384eb097ffor 4 rows
overwriting liquidity_provider address for \x9b33dd59fa401374f9213d591d0319a9d7e9d2cbfor 10 rows
overwriting liquidity_provider address for \xeda61b1e9f414889dde0dbe83b3bacf814a98e87for 2 rows
overwriting liquidity_provider address

In [23]:
display(lp_events.loc[lp_events.token_name == None,:])
none_tx_hashes = []
for _, row in lp_events.iterrows():
    new_row = dict(row)
    token_name = new_row["token_name"]
    if token_name is None:
        # display("warning: none token name:")
        # display(new_row)
        none_tx_hashes.append(new_row["tx_hash"])
display(lp_events.loc[lp_events.tx_hash.isin(none_tx_hashes),:])

,row_num,datetime,tx_hash,address,liquidityProvider,token_name,token_change,usd_change,recipient_address,event_type,expiry_timestamp,expiry_datetime,evt_index,tx_index


,row_num,datetime,tx_hash,address,liquidityProvider,token_name,token_change,usd_change,recipient_address,event_type,expiry_timestamp,expiry_datetime,evt_index,tx_index


In [24]:
display(none_tx_hashes)

[]

In [25]:
sum(lp_events["token_name"] == None)

0

In [26]:
sum(lp_events["tx_hash"].isin(none_tx_hashes))

0

In [27]:
type(address_token_index)

collections.defaultdict

In [28]:
len(address_token_index)

12789

In [29]:
keys = list(address_token_index.keys())
display(len(keys))
display(len(list(set(keys))))

12789

12789

In [30]:
level2 = address_token_index[keys[1]]
level2keys = list(set(level2.keys()))
display(level2keys)

['LPePyvCurveLUSD-27DEC21',
 'LPePyvcrvSTETH-15OCT21',
 'LPeYyvDAI-16OCT21',
 'LPePyvCrvTriCrypto-15AUG21',
 'LPePyvDAI-16OCT21',
 'LPePyvCurve-alUSD-28JAN22',
 'LPeYyvCrvTriCrypto-15AUG21',
 'LPeYyvCurveLUSD-28SEP21',
 'LPeYyvCurveLUSD-27DEC21',
 'LPePyvCurveLUSD-28SEP21']

In [31]:
level2[None]

KeyError: None

In [ ]:
df.shape

In [ ]:
df.index

In [ ]:
df.head(1)